### Importing the Libraries

In [29]:
import pandas as pd
import numpy as np

### Read File names for Download

- First row is taken as the column names.

In [30]:
_file_list = pd.read_csv('Download_file_names.csv')

### Read the blacklist

In [31]:
_category = 'Blacklist'

_blacklist_filenames = _file_list.loc[_file_list['Category']==_category,['File Name','Remarks']]
_blacklist_filenames.reset_index(drop=True,inplace=True)

_refund_blacklist_dict = {}
for ind,i in enumerate(_blacklist_filenames['File Name']):
    _keyname = _blacklist_filenames.loc[ind,'Remarks']
    _table = pd.read_excel(i,skiprows=1,usecols=[0])
    _table['Blacklist Remarks'] = _keyname
    _refund_blacklist_dict[_keyname] = _table

_blacklist = pd.concat(_refund_blacklist_dict,axis=0).reset_index(drop=True)

### Read the Refund List

In [32]:
 _file_list.loc[_file_list['Category']==_category,'File Name']

2    Blacklist1.xlsx
3    Blacklist2.xlsx
Name: File Name, dtype: object

In [33]:
_category = 'Refund List'

_refund_list_filename = _file_list.loc[_file_list['Category']==_category,'File Name'].values[0]
_refund_list = pd.read_excel(_refund_list_filename,skiprows=1)

In [34]:
_refund_list = _refund_list.merge(_blacklist,on='ID',how='left')

_refund_list.sort_values(by=['Amount'],ascending=False,inplace=True)

#_refund_list.to_csv('Output_Refund_Batch_List_Remarks.csv',index=False)

### Fee Report

In [35]:
_category = 'Fee Report'

_fee_report_filename = _file_list.loc[_file_list['Category']==_category,'File Name'].values[0]
_fee_report = pd.read_excel(_fee_report_filename,skiprows=1)

_columns = ['ID',
            'PreValue','Tuition Fee','Non MOE STF','SSG Subsidies/TF Discount',
            'IS16 & CPE1 GST','MMFs','MMFs Discount','Hostel Fee',
            'Scholarship/Bursary','CPF/PSEA/MENDAKI','FA Loan','Payments  Y STUDENT',
            'Refund','ACCOUNT BAL'
           ]

_fee_report = _fee_report[_columns]
_fee_report['Check Acct Bal'] = _fee_report[['PreValue','Tuition Fee','Non MOE STF','SSG Subsidies/TF Discount',
            'IS16 & CPE1 GST','MMFs','MMFs Discount','Hostel Fee',
            'Scholarship/Bursary','CPF/PSEA/MENDAKI','FA Loan','Payments  Y STUDENT',
            'Refund']].sum(axis=1)
_fee_report['Difference'] = _fee_report['Check Acct Bal'] -_fee_report['ACCOUNT BAL']

_fee_report.loc[_fee_report['Difference']>=0.01,'Fee Report Remarks'] = 'Account Balance does not tally. Account Balance supposed to be more.'
_fee_report.loc[_fee_report['Difference']<=-0.01,'Fee Report Remarks'] = 'Account Balance does not tally. Account Balance supposed to be lesser.'

_fee_report_error = _fee_report[_fee_report['Fee Report Remarks'].notnull()].reset_index(drop=True)

In [36]:
#_refund_list = pd.read_csv('Output_Refund_Batch_List_Remarks.csv')

_refund_list = _refund_list.merge(_fee_report_error[['ID','Fee Report Remarks']],on='ID',how='left')

_refund_list.sort_values(by=['Amount'],ascending=False,inplace=True)

#print(_refund_list.head())

_refund_list_dict = {}
_refund_list_dict['Full Refund List'] = _refund_list
_refund_list_dict['Refund List with Remarks'] = _refund_list[~np.logical_and(_refund_list['Blacklist Remarks'].isna(),
                                                                                 _refund_list['Fee Report Remarks'].isna())]

with pd.ExcelWriter('Output_Refund List Summary.xlsx') as writer:
    for key in _refund_list_dict:
        df = pd.DataFrame(_refund_list_dict[key])
        df.to_excel(writer,sheet_name=key,index=False)